### 서포트 벡터 머신
- 선형, 비선형 분류, 회귀, 이상치 탐색에 사용가능한 다목적 머신러닝 모델
- 복잡한 분류 문제나 작거나 중간 크기의 데이터 셋에 적합

##### 선형 SVM 분류
- 라지마진분류를 특성으로 함
- SVM은 특성의 스케일에 민감하므로 Scaler를 사용하면 보다 좋은 결과를 얻을 수 있음

##### 소프트 마진 분류
  - 모든 샘플이 도로 바깥쪽에 분류된다면, 이를 하드 마진 분류라고 함
  - 하지만, 하드마진 분류는 1) 데이터가 선형적으로 구분될 수 있어야 하며, 2) 이상치에 민감하다는 단점이 있음
  - 좀 더 유연한 모델 즉, 도로의 폭을 가능한 넓게 유지하는 것과 마진오류 사이에 적절한 균형이 필요
  - 이를 구현한 것을 '소프트 마진 분류'라고 함
  - parameter 중 C를 감소시키면, SVM모델의 과적합을 규제할 수 있음

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris['data'][:, (2, 3)]
y = (iris['target'] == 2).astype(np.float64)

svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C=1, loss='hinge'))
])

svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('linear_svc', LinearSVC(C=1, loss='hinge'))])

In [2]:
# predict_proba를 사용하려면, probability = True로 매개변수를 지정하면 됨
svm_clf.predict([[5.5, 1.7]])

array([1.])

In [3]:
# 그냥 SVC클래스로도 선형분류 가능 = SVC(kernel='linear', C=1)로 쓰면 됨

##### 비선형 SVM 분류
- 비선형을 다루는 방법 중 하나는 다항특성과 같은 특성을 추가하는 것
- 이를 구현하려면 PolynomialFeatures를 통해 다항특성을 추가시켜주면 됨

In [4]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(100, noise=0.15)
polynomial_svm_clf = Pipeline([
    ('poly_features', PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('svm_clf', LinearSVC(C=10, loss='hinge'))
])
polynomial_svm_clf.fit(X, y)

C:\Users\admin\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('poly_features', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('svm_clf', LinearSVC(C=10, loss='hinge'))])

In [5]:
polynomial_svm_clf.score(X, y)

1.0

##### 다항식 커널
- 다항식 특성을 추가하는 것은 간단하고 유용하지만, 단점도 있음
- 낮은 차수는 복잡한 데이터셋을 설명하지 못하고, 높은 차수는 모델을 느리게 만듦
- SVM은 커널트릭이라는 수학적 기교를 사용해 다항식 특성을 추가한 것과 같은 결과를 만듦

In [6]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel='poly', degree=3, coef0=1, C=5))
])
poly_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=5, coef0=1, kernel='poly'))])

In [7]:
poly_kernel_svm_clf.score(X, y)

1.0

##### 유사도 특성
- 비선형 특성을 다루는 또 다른 기법
- 각 샘플이 특정 랜드마크와 얼마나 닮았는지를 측정하는 유사도 함수로 계산한 특성을 추가하는 것
- ex) 유사도 함수는 = 가우시안 RBF를 통해 종 모양 2개를 만들어 선형적으로 구분하는 것
- n개의 특성을 가진 m개의 샘플이 m개의 특성을 가진 m개의 샘플로 변환됨
- 훈련세트가 매우 클 경우 동일한 크기의 아주 많은 특성이 만들어짐 == 느림
- 마찬가지로 커널트릭을 사용해 속도를 높일 수 있음

In [8]:
# 가우시안 RBF를 활용한 SVC 모델
# gamma를 증가시키면 종모양이 좁아져 각 샘플의 영향 범위가 작아짐(각 샘플을 따라 구불구불하게 휘어짐)
# 반대로, gamma가 작아지면, 넓은 종모양을 그리고 결정경계가 부드러워짐
# 결론적으로 gamma가 규제역할을 함. (과적합일 경우 감소시켜야하며, 과소적합일 경우 증가시켜야 함)

rbf_kernel_svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel='rbf', gamma=5, C=0.001))
])
rbf_kernel_svm_clf.fit(X, y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('svm_clf', SVC(C=0.001, gamma=5))])

In [9]:
rbf_kernel_svm_clf.score(X, y)

0.98

In [10]:
# 대부분의 경우에 가우시안RBF 커널이 선형 SVM보다 효과가 뛰어남
# 하지만 속도가 느리므로 Linear SVM 후, 가우시안 RBF를 사용하는 것이 좋겠음(데이터셋이 작다면 바로 RBF)

##### SVM 회귀
- 선형, 비선형 회귀에 모두 사용될 수 있음
- 제한된 마진오류 안에서 도로 안에 가능한 많은 샘플이 들어가도록 학습
- 스케일을 맞추고, 평균을 0으로 맞춰야 함

In [11]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(epsilon=1.5)

In [12]:
svm_reg.score(X, y)

-1.0

In [14]:
from sklearn.svm import SVR
svm_poly_reg = SVR(kernel='poly', degree=2, C=100, epsilon=.1)
svm_poly_reg.fit(X, y)

SVR(C=100, degree=2, kernel='poly')

In [15]:
svm_poly_reg.score(X,y)

0.2779515103607497